In [1]:
import torch
import logging
import math
import time

from hydra import initialize, compose
from hydra.utils import instantiate
from pytorch_lightning.utilities import move_data_to_device
from bliss.surveys.dc2 import DC2DataModule
from bliss.catalog import TileCatalog
from typing import Dict, Tuple
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence

# set device
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")

# load config
with initialize(config_path=".", version_base=None):
    notebook_cfg = compose("notebook_config")

In [2]:
def to_tile_catalog(
        ori_full_cat,
        tile_slen: int,
        max_sources_per_tile: int,
        ignore_extra_sources=False,
        filter_oob=False,
    ) -> TileCatalog:
        """Returns the TileCatalog corresponding to this FullCatalog.

        Args:
            tile_slen: The side length of the tiles.
            max_sources_per_tile: The maximum number of sources in one tile.
            ignore_extra_sources: If False (default), raises an error if the number of sources
                in one tile exceeds the `max_sources_per_tile`. If True, only adds the tile
                parameters of the first `max_sources_per_tile` sources to the new TileCatalog.
            filter_oob: If filter_oob is true, filter out the sources outside the image. (e.g. In
                case of data augmentation, there is a chance of some sources located outside the
                image)

        Returns:
            TileCatalog correspond to the each source in the FullCatalog.

        Raises:
            ValueError: If the number of sources in one tile exceeds `max_sources_per_tile` and
                `ignore_extra_sources` is False.
            KeyError: If the tile_params contain `plocs` or `n_sources`.
        """
        # TODO: a FullCatalog only needs to "know" its height and width to convert itself to a
        # TileCatalog. So those parameters should be passed on conversion, not initialization.
        tile_coords = torch.div(ori_full_cat["plocs"], tile_slen, rounding_mode="trunc").to(torch.int)
        n_tiles_h = math.ceil(ori_full_cat.height / tile_slen)
        n_tiles_w = math.ceil(ori_full_cat.width / tile_slen)

        # prepare tiled tensors
        tile_cat_shape = (ori_full_cat.batch_size, n_tiles_h, n_tiles_w, max_sources_per_tile)
        tile_locs = torch.zeros((*tile_cat_shape, 2), device=ori_full_cat.device)
        tile_n_sources = torch.zeros(tile_cat_shape[:3], dtype=torch.int64, device=ori_full_cat.device)
        tile_params: Dict[str, Tensor] = {}
        for k, v in ori_full_cat.items():
            if k in {"plocs", "n_sources"}:
                continue
            size = (ori_full_cat.batch_size, n_tiles_h, n_tiles_w, max_sources_per_tile, v.shape[-1])
            tile_params[k] = torch.zeros(size, dtype=v.dtype, device=ori_full_cat.device)

        tile_params["locs"] = tile_locs

        for ii in range(ori_full_cat.batch_size):
            n_sources = int(ori_full_cat["n_sources"][ii].item())
            plocs_ii = ori_full_cat["plocs"][ii][:n_sources]
            filter_sources = n_sources
            source_tile_coords = tile_coords[ii][:n_sources]
            if filter_oob:
                x0_mask = (plocs_ii[:, 0] > 0) & (plocs_ii[:, 0] < ori_full_cat.height)
                x1_mask = (plocs_ii[:, 1] > 0) & (plocs_ii[:, 1] < ori_full_cat.width)
                x_mask = x0_mask * x1_mask
                filter_sources = x_mask.sum()
                source_tile_coords = source_tile_coords[x_mask]

            if filter_sources == 0:
                continue

            source_indices = source_tile_coords[:, 0] * n_tiles_w + source_tile_coords[
                :, 1
            ].unsqueeze(0)
            tile_indices = torch.arange(n_tiles_h * n_tiles_w, device=ori_full_cat.device).unsqueeze(1)

            tile_to_source_mapping = (source_indices == tile_indices).nonzero()
            tile_source_count: Tuple[Tensor, Tensor] = tile_to_source_mapping[:, 0].unique(
                sorted=True, return_counts=True
            )  # first element is tile index; second element is source count
            if tile_source_count[1].max() > max_sources_per_tile:
                if not ignore_extra_sources:
                    raise ValueError(  # noqa: WPS220
                        "# of sources per tile exceeds `max_sources_per_tile`."
                    )

            # get n_sources for each tile
            tile_n_sources[ii].view(-1)[tile_source_count[0].flatten().tolist()] = torch.where(
                tile_source_count[1] <= max_sources_per_tile,
                tile_source_count[1],
                max_sources_per_tile,
            )

            for k, v in tile_params.items():
                if k == "plocs":
                    raise KeyError("plocs should not be in tile_params")
                if k == "locs":
                    k = "plocs"
                if k == "n_sources":
                    raise KeyError("n_sources should not be in tile_params")
                param_matrix = ori_full_cat[k][ii][:n_sources]
                if filter_oob:
                    param_matrix = param_matrix[x_mask]
                params_on_tile = list(
                    param_matrix[tile_to_source_mapping[:, 1]].split(
                        tile_source_count[1].flatten().tolist()
                    )
                )
                # pad first tensor to desired length
                # the second argument of pad function is
                # padding_left, padding_right, padding_top, padding_bottom
                params_on_tile[0] = torch.nn.functional.pad(
                    params_on_tile[0],
                    (0, 0, 0, (max_sources_per_tile - params_on_tile[0].shape[0])),
                )
                # pad all tensors
                params_on_tile = pad_sequence(params_on_tile, batch_first=True)
                max_fill = min(filter_sources, max_sources_per_tile)
                v[ii].view(-1, *v[ii].shape[2:])[
                    tile_to_source_mapping[:, 0].unique(sorted=True).tolist(), :max_fill
                ] = params_on_tile[:, :max_fill].to(dtype=v.dtype)

            # modify tile location
            tile_params["locs"][ii] = (tile_params["locs"][ii] % tile_slen) / tile_slen
        tile_params.update({"n_sources": tile_n_sources})
        return TileCatalog(tile_params)

In [3]:
def test_tile_cat_equal(left_tile_cat, right_tile_cat):
    logger = logging.getLogger(__name__)
    is_equal = True
    for k, v in left_tile_cat.items():
        cur_test_equal = torch.allclose(right_tile_cat[k], v, equal_nan=True)
        if not cur_test_equal:
            logger.warning("%s are different", k)
        is_equal &= cur_test_equal
    return is_equal

In [4]:
# setup bliss encoder
tile_slen = notebook_cfg.surveys.dc2.tile_slen
max_sources_per_tile = notebook_cfg.surveys.dc2.max_sources_per_tile

dc2: DC2DataModule = instantiate(notebook_cfg.surveys.dc2)
dc2.setup(stage="validate")
dc2_val_dataloader = dc2.val_dataloader()

In [5]:
old_gpu_time = 0
new_gpu_time = 0
for i, batch in enumerate(dc2_val_dataloader):
    batch_on_device = move_data_to_device(batch, device=device)
    full_cat = TileCatalog(batch_on_device["tile_catalog"]).to_full_catalog(tile_slen)

    old_start_time = time.time()
    old_tile_cat = to_tile_catalog(full_cat, tile_slen=tile_slen, max_sources_per_tile=max_sources_per_tile)
    old_end_time = time.time()
    print(f"[{i}] old: {old_end_time - old_start_time: .3f}")
    old_gpu_time += (old_end_time - old_start_time)

    new_start_time = time.time()
    new_tile_cat = full_cat.to_tile_catalog(tile_slen=tile_slen, max_sources_per_tile=max_sources_per_tile)
    new_end_time = time.time()
    print(f"[{i}] new: {new_end_time - new_start_time: .3f}")
    new_gpu_time += (new_end_time - new_start_time)

    assert test_tile_cat_equal(old_tile_cat, new_tile_cat)

[0] old:  0.976
[0] new:  0.112
[1] old:  0.384
[1] new:  0.009
[2] old:  0.569
[2] new:  0.008
[3] old:  0.395
[3] new:  0.013
[4] old:  0.765
[4] new:  0.011
[5] old:  0.753
[5] new:  0.011
[6] old:  0.745
[6] new:  0.010
[7] old:  0.737
[7] new:  0.010
[8] old:  0.742
[8] new:  0.009
[9] old:  0.724
[9] new:  0.011
[10] old:  0.432
[10] new:  0.006
[11] old:  0.741
[11] new:  0.010
[12] old:  0.738
[12] new:  0.012
[13] old:  0.542
[13] new:  0.006
[14] old:  0.539
[14] new:  0.011
[15] old:  0.737
[15] new:  0.010
[16] old:  0.736
[16] new:  0.010
[17] old:  0.760
[17] new:  0.011
[18] old:  0.767
[18] new:  0.012
[19] old:  0.757
[19] new:  0.011
[20] old:  0.730
[20] new:  0.011
[21] old:  0.748
[21] new:  0.011
[22] old:  0.850
[22] new:  0.012
[23] old:  0.636
[23] new:  0.005
[24] old:  0.476
[24] new:  0.007
[25] old:  0.427
[25] new:  0.007
[26] old:  0.772
[26] new:  0.010
[27] old:  0.756
[27] new:  0.011
[28] old:  0.765
[28] new:  0.011
[29] old:  0.761
[29] new:  0.010


In [6]:
print(f"old_gpu_time: {old_gpu_time: .3f}")
print(f"new_gpu_time: {new_gpu_time: .3f}")
print(f"old/new: {old_gpu_time / new_gpu_time: .3f}")

old_gpu_time:  249.030
new_gpu_time:  3.548
old/new:  70.186


In [ ]:
old_cpu_time = 0
new_cpu_time = 0
for i, batch in enumerate(dc2_val_dataloader):
    batch_on_device = move_data_to_device(batch, device="cpu")
    full_cat = TileCatalog(batch_on_device["tile_catalog"]).to_full_catalog(tile_slen)

    old_start_time = time.time()
    old_tile_cat = to_tile_catalog(full_cat, tile_slen=tile_slen, max_sources_per_tile=max_sources_per_tile)
    old_end_time = time.time()
    print(f"[{i}] old: {old_end_time - old_start_time: .3f}")
    old_cpu_time += (old_end_time - old_start_time)

    new_start_time = time.time()
    new_tile_cat = full_cat.to_tile_catalog(tile_slen=tile_slen, max_sources_per_tile=max_sources_per_tile)
    new_end_time = time.time()
    print(f"[{i}] new: {new_end_time - new_start_time: .3f}")
    new_cpu_time += (new_end_time - new_start_time)

    assert test_tile_cat_equal(old_tile_cat, new_tile_cat)

In [ ]:
print(f"old_cpu_time: {old_cpu_time: .3f}")
print(f"new_cpu_time: {new_cpu_time: .3f}")
print(f"old/new: {old_cpu_time / new_cpu_time: .3f}")

In [7]:
full_image_old_time = 0
full_image_new_time = 0
for i in range(10):
    full_cat = dc2.get_plotting_sample(i)["full_catalog"]

    old_start_time = time.time()
    old_tile_cat = to_tile_catalog(full_cat, tile_slen, max_sources_per_tile)
    old_end_time = time.time()
    print(f"[{i}] old: {old_end_time - old_start_time: .3f}")
    full_image_old_time += (old_end_time - old_start_time)

    new_start_time = time.time()
    new_tile_cat = full_cat.to_tile_catalog(tile_slen, max_sources_per_tile)
    new_end_time = time.time()
    print(f"[{i}] new: {new_end_time - new_start_time: .3f}")
    full_image_new_time += (new_end_time -new_start_time)

    assert test_tile_cat_equal(old_tile_cat, new_tile_cat)

[0] old:  30.555
[0] new:  0.952
[1] old:  25.015
[1] new:  1.215
[2] old:  27.132
[2] new:  1.120
[3] old:  21.124
[3] new:  0.972
[4] old:  25.088
[4] new:  0.881
[5] old:  25.066
[5] new:  0.924
[6] old:  25.095
[6] new:  0.861
[7] old:  28.048
[7] new:  1.775
[8] old:  22.203
[8] new:  1.048
[9] old:  21.952
[9] new:  1.092


In [8]:
print(f"full_image_old_time: {full_image_old_time: .3f}")
print(f"full_image_new_time: {full_image_new_time: .3f}")
print(f"old/new: {full_image_old_time / full_image_new_time: .3f}")

full_image_old_time:  251.277
full_image_new_time:  10.839
old/new:  23.183
